In [1]:
import logging

# Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create a console handler
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# Add the formatter to the handler
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

In [2]:
import pandas as pd

## Load the datasets

In [19]:
# We downloaded the dataset in chunks from https://www.smard.de/home/downloadcenter/download-marktdaten
data = pd.concat([
    pd.read_csv('../data/raw/2015_2016.csv', delimiter=";"),
    pd.read_csv('../data/raw/2017_2018.csv', delimiter=";"),
    pd.read_csv('../data/raw/2019_2020.csv', delimiter=";"),
    pd.read_csv('../data/raw/2021_2022.csv', delimiter=";"),
    pd.read_csv('../data/raw/2022_2023.csv', delimiter=";"),
], ignore_index=True)

logger.info(f'Data loading done.')

2023-11-11 02:18:25,236 - INFO - Data loading done.


## Preprocessing
The data does not require that much data cleaning. We change the interval based way to track the time in the original SMARD dataset to single timestamps, which is easier to process. In the original data we have a energy mix for a given start and end time, which we transformed to a timestamp of **just** the start time i.e. the energy mix with start time 07:00 and end time 08:00 is now denoted as the energy mix at timestamp 07:00.

We choose **31.03.2023** as the cut-off date for our dataset.

In [20]:
# Function to convert German number notation to float
def convert_to_float(german_number_str):
    try:
        return float(german_number_str.replace('.', '').replace(',', '.'))
    except ValueError:
        return None

# Convert "Datum" column to datetime
data['Datum'] = pd.to_datetime(data['Datum'], format='%d.%m.%Y')

# Convert "Anfang" and "Ende" columns to time
data['Anfang'] = pd.to_datetime(data['Anfang'], format='%H:%M').dt.time
data['Ende'] = pd.to_datetime(data['Ende'], format='%H:%M').dt.time

# Convert energy columns to float
energy_columns = [col for col in data.columns if 'MWh' in col]
for col in energy_columns:
    data[col] = data[col].apply(convert_to_float)
logger.info("Converted energy columns to float.")

# Combine "Datum" and "Anfang" into a single datetime column
data['Timestamp'] = pd.to_datetime(data['Datum'].astype(str) + ' ' + data['Anfang'].astype(str))

# Drop the "Datum", "Anfang", and "Ende" columns
data = data.drop(columns=['Datum', 'Anfang', 'Ende'])

# Reorder columns to have "Timestamp" as the first column
data = data[['Timestamp'] + [col for col in data.columns if col != 'Timestamp']]

# Define new column names
new_column_names = {
    'Timestamp': 'timestamp',
    'Biomasse [MWh] Berechnete Auflösungen': 'biomass_mwh',
    'Wasserkraft [MWh] Berechnete Auflösungen': 'hydropower_mwh',
    'Wind Offshore [MWh] Berechnete Auflösungen': 'wind_offshore_mwh',
    'Wind Onshore [MWh] Berechnete Auflösungen': 'wind_onshore_mwh',
    'Photovoltaik [MWh] Berechnete Auflösungen': 'photovoltaic_mwh',
    'Sonstige Erneuerbare [MWh] Berechnete Auflösungen': 'other_renewables_mwh',
    'Kernenergie [MWh] Berechnete Auflösungen': 'nuclear_mwh',
    'Braunkohle [MWh] Berechnete Auflösungen': 'brown_coal_mwh',
    'Steinkohle [MWh] Berechnete Auflösungen': 'hard_coal_mwh',
    'Erdgas [MWh] Berechnete Auflösungen': 'natural_gas_mwh',
    'Pumpspeicher [MWh] Berechnete Auflösungen': 'pumped_storage_mwh',
    'Sonstige Konventionelle [MWh] Berechnete Auflösungen': 'other_conventional_mwh'
}

# Rename the columns
data.rename(columns=new_column_names, inplace=True)

# Cut off the data at 2023-03-31
data = data.loc[data['timestamp'] <= '2023-03-31']

# Check for missing values
missing_values = data.isnull().sum()

# Log the missing values
logger.info(f'Missing values in each column:\n{missing_values}')

# Some timestamps are duplicates, remove them
data = data.drop_duplicates(subset='timestamp')

# Reindex the DataFrame
data = data.reset_index(drop=True)

# Pytorch throws an error on M1/M2 Macbooks with float64
for column in data.select_dtypes(include=['float64']).columns:
    data[column] = data[column].astype('float32')

# Display the first few rows of the modified dataframe
logger.info('Data preprocessing done.')
data

2023-11-11 02:19:41,040 - INFO - Converted energy columns to float.
2023-11-11 02:19:41,124 - INFO - Missing values in each column:
timestamp                 0
biomass_mwh               0
hydropower_mwh            0
wind_offshore_mwh         0
wind_onshore_mwh          0
photovoltaic_mwh          0
other_renewables_mwh      0
nuclear_mwh               0
brown_coal_mwh            0
hard_coal_mwh             0
natural_gas_mwh           0
pumped_storage_mwh        0
other_conventional_mwh    0
dtype: int64
2023-11-11 02:19:41,135 - INFO - Data preprocessing done.


,timestamp,biomass_mwh,hydropower_mwh,wind_offshore_mwh,wind_onshore_mwh,photovoltaic_mwh,other_renewables_mwh,nuclear_mwh,brown_coal_mwh,hard_coal_mwh,natural_gas_mwh,pumped_storage_mwh,other_conventional_mwh
0,2015-01-01 00:00:00,4024.25,1158.25,516.50,8128.00,0.00,133.00,10710.50,15687.25,3219.75,1226.25,1525.75,4909.25
1,2015-01-01 01:00:00,3982.75,1188.00,516.25,8297.50,0.00,122.50,11086.25,15321.75,2351.25,870.75,1079.25,4932.75
2,2015-01-01 02:00:00,4019.50,1139.25,514.00,8540.00,0.00,93.00,11026.25,14817.50,2227.00,809.50,787.00,5041.75
3,2015-01-01 03:00:00,4040.75,1122.50,517.75,8552.00,0.00,86.50,11027.75,14075.00,2339.75,821.00,287.75,5084.00
4,2015-01-01 04:00:00,4037.75,1112.00,519.75,8643.50,0.00,86.50,10962.25,14115.00,2461.50,831.25,346.75,5070.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72251,2023-03-30 20:00:00,4834.25,1640.00,5335.25,25581.50,0.25,141.25,2789.25,9220.25,3644.25,5144.50,4806.50,1347.50
72252,2023-03-30 21:00:00,4731.75,1700.00,5307.50,25894.75,0.00,141.00,2794.25,8643.50,2833.00,4897.50,2961.25,1329.00
72253,2023-03-30 22:00:00,4596.75,1734.25,5360.25,25610.50,0.00,135.25,2787.50,7781.75,2655.00,4224.50,1407.25,1323.75
72254,2023-03-30 23:00:00,4513.25,1714.75,4675.50,24776.75,0.00,134.25,2787.00,6799.50,2587.00,3816.00,600.25,1333.00


In [10]:
# Save the dataset as parquet file (compared a csv file, this keeps column datatypes)
data.to_parquet("../data/processed/energy_data_processed.parquet")

In [11]:
# Load the exact preprocessed dataset
data = pd.read_parquet("../data/processed/energy_data_processed.parquet")

,timestamp,biomass_mwh,hydropower_mwh,wind_offshore_mwh,wind_onshore_mwh,photovoltaic_mwh,other_renewables_mwh,nuclear_mwh,brown_coal_mwh,hard_coal_mwh,natural_gas_mwh,pumped_storage_mwh,other_conventional_mwh
0,2015-01-01 00:00:00,4024.25,1158.25,516.50,8128.00,0.00,133.00,10710.50,15687.25,3219.75,1226.25,1525.75,4909.25
1,2015-01-01 01:00:00,3982.75,1188.00,516.25,8297.50,0.00,122.50,11086.25,15321.75,2351.25,870.75,1079.25,4932.75
2,2015-01-01 02:00:00,4019.50,1139.25,514.00,8540.00,0.00,93.00,11026.25,14817.50,2227.00,809.50,787.00,5041.75
3,2015-01-01 03:00:00,4040.75,1122.50,517.75,8552.00,0.00,86.50,11027.75,14075.00,2339.75,821.00,287.75,5084.00
4,2015-01-01 04:00:00,4037.75,1112.00,519.75,8643.50,0.00,86.50,10962.25,14115.00,2461.50,831.25,346.75,5070.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72251,2023-03-30 20:00:00,4834.25,1640.00,5335.25,25581.50,0.25,141.25,2789.25,9220.25,3644.25,5144.50,4806.50,1347.50
72252,2023-03-30 21:00:00,4731.75,1700.00,5307.50,25894.75,0.00,141.00,2794.25,8643.50,2833.00,4897.50,2961.25,1329.00
72253,2023-03-30 22:00:00,4596.75,1734.25,5360.25,25610.50,0.00,135.25,2787.50,7781.75,2655.00,4224.50,1407.25,1323.75
72254,2023-03-30 23:00:00,4513.25,1714.75,4675.50,24776.75,0.00,134.25,2787.00,6799.50,2587.00,3816.00,600.25,1333.00
